In [10]:
# Converts Fermate xls to csv
import os
import pandas as pd

for l in os.listdir("dataset/fermi_excel/Fermate"):
    df = pd.read_excel(f"dataset/fermi_excel/Fermate/{l}")
    name = l.replace(".xls", "")

    df.to_csv(f"dataset/fermi/Fermate/{name}.csv", index=False)

df = pd.read_excel(f"dataset/fermi_excel/FERMATE 2211 ACR.xls")
df.to_csv(f"dataset/fermi/FERMATE 2211 ACR.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/fermi_excel/Fermate'

Now I iterate for every machine to get the data from the 3 datasets and merge them into one dataset, removing the ones that are not useful for the analysis.

IT1 ~ Now we can check for all the dataset and see what we can do with them

In [ ]:
# Merge the 3 dataset into one and saves in it1
import os

import pandas as pd
from scripts.getDataset import getEntireDataset, getList

DEBUG = True
pd.set_option("display.max_rows", None)
pd.options.mode.copy_on_write = False

toskip = [file for file in os.listdir("dataset/results/it1")]

complete_dataset = getList()

for machineId, year, month in complete_dataset:
    filename = f"id-{machineId}_{year}-{month}.csv"

    if filename in toskip:
        print("file already exists", filename)
        continue

    dataset = getEntireDataset(machineId, year, month, False)

    dataset.to_csv(
        f"dataset/results/it1/{filename}", index=False
    )


file already exists id-306_2023-1.csv
file already exists id-306_2023-2.csv
file already exists id-306_2023-3.csv
file already exists id-306_2023-4.csv
file already exists id-306_2023-5.csv
file already exists id-306_2023-6.csv
file already exists id-306_2023-7.csv
file already exists id-306_2023-8.csv
file already exists id-306_2023-9.csv
file already exists id-306_2023-10.csv
file already exists id-306_2023-11.csv
file already exists id-306_2023-12.csv
file already exists id-306_2022-3.csv
file already exists id-306_2022-5.csv
file already exists id-306_2022-6.csv
file already exists id-306_2022-7.csv
file already exists id-306_2022-8.csv
file already exists id-306_2022-9.csv
file already exists id-306_2022-10.csv
file already exists id-306_2022-11.csv
file already exists id-306_2022-12.csv
file already exists id-305_2022-3.csv
file already exists id-305_2022-5.csv
file already exists id-305_2022-6.csv
file already exists id-305_2022-7.csv
file already exists id-305_2022-8.csv
file a

IT2 ~ It takes 25 minutes to compute everything but now I want to look at the lifetime of the machines and see if I can find something interesting, merging the data into one file for machine

In [ ]:
# Merge data time series into a single file for each machine, also append material
import pandas as pd
import os
from scripts.getSingleDataset.utils import getCleanDataset, replaceWithUnknown
from scripts.getSingleDataset.getProductions import getProductionWithFixedComma

# get materials
materials = getProductionWithFixedComma(f"dataset/fermi/FERMATE 2211 ACR.csv")[
    ["CODART", "ACR"]
].drop_duplicates()
materials.rename(columns={"CODART": "COD_ART", "ACR": "Material"}, inplace=True)

complete_dataset = {}

for path in os.listdir("dataset/results/it1"):
    splitted_filename = [p.split("-") for p in path.replace(".csv", "").split("_")]

    try:
        d = getCleanDataset(f"dataset/results/it1/{path}")
    except pd.errors.EmptyDataError as e:
        print("Skipping file is probably empty", path)
        continue

    if d.empty:
        print("Skipping as empty", path)
        continue

    machineId = int(splitted_filename[0][1])

    d = pd.merge(d, materials, how="left")

    v = d[d["START_DATE"] == pd.to_datetime("2022-05-30 10:30")]
    
    if v.count().sum() > 0:
        print(v)

    value = [d]

    if machineId in complete_dataset:
        old_data = complete_dataset.get(machineId)

        for old in old_data:
            value.append(old)

    complete_dataset.update({machineId: value})

for machineId in complete_dataset.keys():
    df = pd.concat(complete_dataset.get(machineId))
    prev_col = df.columns

    df = (
        df.groupby(["START_DATE", "END_DATE"])
        .agg({
            "Stop": "first",
            "COD_ART": "first",
            "Productions": "sum",
            "EnergyConsumption": "sum",
            "Material": "first",
        })
        .reset_index()
    )
    assert (prev_col == df.columns).all(), (prev_col, df.columns)

    df = replaceWithUnknown(df)

    size = df.shape[0]
    df.drop_duplicates(inplace=True)
    assert size == df.shape[0], df.shape[0]

    df.to_csv(f"dataset/results/it2/id-{machineId}.csv", mode="w", index=False)

Skipping file is probably empty id-315_2022-3.csv
Skipping file is probably empty id-610_2022-9.csv
Skipping file is probably empty id-108_2022-10.csv
Skipping file is probably empty id-611_2022-10.csv
Skipping file is probably empty id-110_2022-9.csv
Skipping file is probably empty id-309_2022-3.csv
Skipping file is probably empty id-313_2022-9.csv
Skipping file is probably empty id-303_2022-10.csv
Skipping file is probably empty id-611_2023-4.csv
Skipping file is probably empty id-618_2022-10.csv
Skipping file is probably empty id-302_2022-9.csv
Skipping file is probably empty id-303_2023-4.csv
Skipping file is probably empty id-304_2022-3.csv
Skipping file is probably empty id-313_2022-10.csv
Skipping file is probably empty id-618_2022-9.csv
Skipping file is probably empty id-307_2022-9.csv
Skipping file is probably empty id-308_2023-8.csv
Skipping file is probably empty id-515_2022-3.csv
Skipping file is probably empty id-308_2023-9.csv
Skipping file is probably empty id-301_2022-3

We can show the results to make the analysis

In [ ]:
# Plot the data
import os
from scripts.plots import plot, correlation_plot
from scripts.getSingleDataset.utils import getCleanDataset
from scripts.getSingleDataset.utils import takeRange


for file in os.listdir("dataset/results/it2"):
    df = getCleanDataset(f"dataset/results/it2/{file}")
    machineId = int(file.replace(".csv", "").split("-")[1])

    df = takeRange(df, 5)
    plot(df, machineId)
    # correlation_plot(df, machineId)

    # if "x" == input("Press Enter to continue, x to close..."):
    #     break

Now we will use a Machine learning model to predict when a machine will stop

In [15]:
# Classic ML analysis

import os
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
pd.set_option("display.max_rows", None)

SHIFT_VALUE = 2
BASE_COLUMNS_TO_SHIFT = {
    "Productions": "Productions_prev",
    "EnergyConsumption": "EnergyConsumption_prev"
}

shift_columns = [
    f"{c}_{s}"
    for c in BASE_COLUMNS_TO_SHIFT.values()
    for s in range(1, SHIFT_VALUE + 1)
]

COLUMNS = [
    "MachineId",
    "Productions",
    "EnergyConsumption",
    "COD_ART_HASH",
    "Material_HASH",
] + shift_columns


train_data = pd.DataFrame()
for file in os.listdir("dataset/results/it2"):
    data = pd.read_csv(f"dataset/results/it2/{file}")

    data.sort_values(by="START_DATE", inplace=True)

    data["Material_HASH"] = data["Material"].apply(hash)

    data["COD_ART_HASH"] = data["COD_ART"].apply(hash)

    data["MachineId"] = int(file.replace(".csv", "").split("-")[1])

    data.dropna()

    previous_size = data.shape[0]

    for s in range(1, SHIFT_VALUE + 1):
        for c in BASE_COLUMNS_TO_SHIFT.keys():
            data[f"{BASE_COLUMNS_TO_SHIFT[c]}_{s}"] = data[c].shift(s)

    assert previous_size == data.shape[0]

    train_data = pd.concat([train_data, data])


# train_data[:10000].to_csv("dataset/results/complete.csv", mode="w", index=False)
# print(train_data.columns)

def calculateModel(dataset: pd.DataFrame, model: HistGradientBoostingClassifier):
    print("\tTotal rows:", dataset.shape[0])
    dataset["Stop_index"] = dataset["Stop"].apply(lambda x: 1 if x == "Running" else 0)

    rows = int(dataset.shape[0] * 0.6)
    train_d = dataset[:rows]
    test_d = dataset[rows:]

    if train_d["Stop"].value_counts()["Running"] == train_d.shape[0]:
        raise Exception("Data is not correctly distributed train have only 'Running'")
    if test_d["Stop"].value_counts()["Running"] == test_d.shape[0]:
        raise Exception("Data is not correctly distributed test have only 'Running'")

    if train_d.shape[0] == 0 or test_d.shape[0] == 0:
        raise Exception(
            f"\tNot enough data, dataset: {dataset.shape[0]},  train: {train_d.shape[0]}, test: {test_d.shape[0]}"
        )

    print("\tTraining rows:", train_d.shape[0])

    model.fit(train_d[COLUMNS], train_d["Stop_index"])

    # Valutare il modello utilizzando i dati di test
    prediction = model.predict(test_d[COLUMNS])

    df = pd.DataFrame(
        {
            "Prediction": prediction,
            "Real": test_d["Stop"],
            "Correct": prediction == test_d["Stop_index"],
        }
    )

    assert df.shape[0] == test_d.shape[0]

    if df.shape[0] == 0:
        raise Exception("Dataset is not usable")

    # this is only useful to remove the "Running" values from the calculations of the percentile
    # print(df.drop_duplicates())
    # print(df[df["Real"] != "Running"])
    df = df[df["Real"] != "Running"]

    if df.shape[0] == 0:
        raise Exception("No data to calculate")

    res = df["Real"].value_counts()

    errors = res[False] if False in res else 0
    percentile = int(errors / df.shape[0] * 100)

    print(f"\tPrediction errors {percentile}%")
    if percentile == 0:
        print("---------")
        print(errors)#, df.shape[0])

        print(df[df["Correct"] == True].count())
        print(res)

    return percentile

model = HistGradientBoostingClassifier()

average = 0
usables = 0
for machine_id in train_data["MachineId"].unique():
    print("Machine", machine_id)

    dataset = train_data[train_data["MachineId"] == machine_id]

    try:
        dataset = dataset.dropna()
        average += calculateModel(dataset, model)
        usables += 1
    except Exception as e:
        print("\tError:", e)
if usables == 0:
    print("No usable dataset found")
else:
    average = average / usables
    print(f"Avg Error: {average:.2f}%")

print("--- Complete model ---")
print(f"Error: {calculateModel(train_data, model)}%")

Machine 304
	Total rows: 25110
	Error: Data is not correctly distributed test have only 'Running'
Machine 310
	Total rows: 25676
	Error: Data is not correctly distributed test have only 'Running'
Machine 305
	Total rows: 15528
	Error: Data is not correctly distributed test have only 'Running'
Machine 515
	Total rows: 24010
	Error: Data is not correctly distributed test have only 'Running'
Machine 313
	Total rows: 25579
	Error: Data is not correctly distributed test have only 'Running'
Machine 307
	Total rows: 25537
	Error: Data is not correctly distributed test have only 'Running'
Machine 110
	Total rows: 20015
	Error: Data is not correctly distributed test have only 'Running'
Machine 306
	Total rows: 25527
	Error: Data is not correctly distributed test have only 'Running'
Machine 302
	Total rows: 25490
	Error: Data is not correctly distributed test have only 'Running'
Machine 303
	Total rows: 25523
	Error: Data is not correctly distributed test have only 'Running'
Machine 301
	Total r

Now I will use PySpark to implement MlLib and train a model to predict when a machine will stop

In [1]:
# Big Data ML analysis
import os
import pandas as pd

from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import when, isnan, isnull

pd.set_option("display.max_rows", None)

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")


SHIFT_VALUE = 4
BASE_COLUMNS_TO_SHIFT = {
    "Productions": "Productions_prev",
    "EnergyConsumption": "EnergyConsumption_prev",
    "Material_HASH": "Material_Previous",
    "COD_ART_HASH": "COD_ART_Previous",
}

shift_columns = [
    f"{c}_{s}"
    for c in BASE_COLUMNS_TO_SHIFT.values()
    for s in range(1, SHIFT_VALUE + 1)
]

COLUMNS = [
    "MachineId",
    "Productions",
    "EnergyConsumption",
    "COD_ART_HASH",
    "Material_HASH",
] + shift_columns

train_data = pd.DataFrame()
for file in os.listdir("dataset/results/it2"):
    data = pd.read_csv(f"dataset/results/it2/{file}")

    data.sort_values(by="START_DATE", inplace=True)

    data["Material_HASH"] = data["Material"].apply(hash)

    data["COD_ART_HASH"] = data["COD_ART"].apply(hash)

    data["Stop_index"] = data["Stop"].apply(lambda x: 1 if x == "Running" else 0)

    data["MachineId"] = int(file.replace(".csv", "").split("-")[1])

    data.dropna()

    previous_size = data.shape[0]

    for s in range(1, SHIFT_VALUE + 1):
        for c in BASE_COLUMNS_TO_SHIFT.keys():
            data[f"{BASE_COLUMNS_TO_SHIFT[c]}_{s}"] = data[c].shift(s)

    assert previous_size == data.shape[0]

    train_data = pd.concat([train_data, data])

train_data.to_csv("dataset/results/complete.csv", mode="w", index=False)

assembler = VectorAssembler(inputCols=COLUMNS,outputCol="features", handleInvalid="keep")
labelIndexer = StringIndexer(
    inputCol="Stop_index", outputCol="indexedLabel", handleInvalid="keep"
)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4, handleInvalid="keep")
gbt = GBTClassifier(labelCol="indexedLabel",featuresCol="indexedFeatures",maxIter=10)
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy"
)

# MlLib
def calculateModel(dataset: pd.DataFrame):
    dataset = spark.createDataFrame(dataset)

    print("\tTotal rows:", dataset.count())

    for col in dataset.columns:
        dataset = dataset.withColumn(
            col, when(isnan(dataset[col]), 0).otherwise(dataset[col])
        )

    dataset = assembler.transform(dataset)
    dataset = dataset.na.fill(0)

    train_d, test_d = dataset.randomSplit([0.8, 0.2])

    if train_d.count() == 0 or test_d.count() == 0:
        raise Exception(
            "Not enough data", dataset.count(), train_d.count(), test_d.count()
        )

    pipeline = Pipeline(
        stages=[labelIndexer.fit(train_d), featureIndexer.fit(train_d), gbt],
    )

    model = pipeline.fit(train_d)

    prediction = model.transform(test_d)

    accuracy = evaluator.evaluate(prediction)

    percentile = (1-accuracy) * 100
    print(f"\tPrediction errors {percentile:.4f}%")

    return percentile

average = 0
usables = 0
for machine_id in train_data["MachineId"].unique():
    print("Machine", machine_id)

    dataset = train_data[train_data["MachineId"] == machine_id]

    try:
        average += calculateModel(dataset)
        usables += 1
    except Exception as e:
        print("\t", e)

if usables == 0:
    print("No usable data")
else:
    average = average / usables
    print(f"Avg Error: {average:.2f}%")

# print(f"Complete model\n\tError: {calculateModel(train_data)}%")
    

24/03/20 14:19:05 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.25.147 instead (on interface en0)
24/03/20 14:19:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/20 14:19:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Machine 304


	Total rows: 25112
	Prediction errors 0.0399%
Machine 310
	Total rows: 25678
	Prediction errors 0.0389%
Machine 305
	Total rows: 15530
	Prediction errors 0.0958%
Machine 515
	Total rows: 25542
	Prediction errors 0.0197%
Machine 313
	Total rows: 25581
	Prediction errors 0.0000%
Machine 307
	Total rows: 25539
	Prediction errors 0.0586%
Machine 110
	Total rows: 20017
	Prediction errors 0.0993%
Machine 306
	Total rows: 25529
	Prediction errors 0.0201%
Machine 302
	Total rows: 25492
	Prediction errors 0.0000%
Machine 303
	Total rows: 25525
	Prediction errors 0.0586%
Machine 301
	Total rows: 25515
	Prediction errors 0.0196%
Machine 315
	Total rows: 25552
	Prediction errors 0.0390%
Machine 314
	Total rows: 25596
	Prediction errors 0.0193%
Machine 614
	Total rows: 25575
	Prediction errors 0.1171%
Machine 611
	Total rows: 25534
	Prediction errors 0.0984%
Machine 610
	Total rows: 25613
	Prediction errors 0.0000%
Machine 612
	Total rows: 25584
	Prediction errors 0.0398%
Machine 618
	Total rows: 2